# 🔗 JSON-LD Export & W3C Verifiable Credentials

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/artiso-ai/dppvalidator/blob/main/examples/notebooks/04_jsonld_export.ipynb)

Learn how to export validated DPPs as W3C Verifiable Credentials in JSON-LD format.

This notebook covers:
- Understanding DPP as Verifiable Credentials
- JSON-LD context and namespaces
- Exporting validated passports

In [ ]:
!pip install -q dppvalidator

## 📋 Understanding DPP Structure

A Digital Product Passport follows the W3C Verifiable Credentials Data Model:

```
DPP = Verifiable Credential
├── @context (JSON-LD namespaces)
├── type (DigitalProductPassport, VerifiableCredential)
├── id (unique credential identifier)
├── issuer (organization issuing the passport)
├── validFrom / validUntil (validity period)
└── credentialSubject (product information)
```

## 🏗️ Creating a Valid DPP

In [ ]:
from dppvalidator.validators import ValidationEngine

engine = ValidationEngine()

dpp_data = {
    "type": ["DigitalProductPassport", "VerifiableCredential"],
    "@context": [
        "https://www.w3.org/ns/credentials/v2",
        "https://test.uncefact.org/vocabulary/untp/dpp/0.6.1/",
    ],
    "id": "https://example.com/credentials/dpp-organic-cotton-001",
    "issuer": {
        "id": "did:web:example.com:issuers:sustainable-fashion",
        "name": "Sustainable Fashion Co.",
        "type": ["Organization"],
    },
    "validFrom": "2024-01-01T00:00:00Z",
    "validUntil": "2027-01-01T00:00:00Z",
    "credentialSubject": {
        "type": ["Product"],
        "id": "https://example.com/products/organic-cotton-tshirt-001",
        "name": "Organic Cotton Premium T-Shirt",
        "description": "GOTS certified organic cotton t-shirt",
        "granularityLevel": "model",
        "product": {
            "type": ["Product"],
            "id": "https://example.com/products/organic-cotton-tshirt-001",
            "name": "Organic Cotton Premium T-Shirt",
            "productCategory": ["Textiles", "Apparel"],
        },
        "materialsProvenance": [
            {
                "type": ["Material"],
                "name": "GOTS Organic Cotton",
                "massFraction": 1.0,
                "recycled": False,
                "hazardous": False,
            }
        ],
        "circularityScorecard": {
            "type": ["CircularityScorecard"],
            "recycledContent": 0.0,
            "recyclableContent": 0.98,
            "repairabilityScore": 8,
        },
        "conformityClaim": [
            {
                "type": ["Claim"],
                "claimType": "Certification",
                "topic": "sustainability",
                "conformityEvidence": {
                    "type": ["Evidence"],
                    "evidenceType": "Certificate",
                    "linkURL": "https://example.com/certificates/gots-2024-001",
                },
            }
        ],
    },
}

result = engine.validate(dpp_data)
print(f"✓ Valid: {result.valid}")
print(f"  Errors: {result.error_count}")
print(f"  Warnings: {result.warning_count}")

## 📤 Exporting as JSON-LD

In [ ]:
from dppvalidator.exporters import JSONLDExporter

exporter = JSONLDExporter()

# Export validated passport to JSON-LD
if result.passport:
    jsonld_output = exporter.export(result.passport)
    print("JSON-LD Output:")
    print(jsonld_output[:500] + "...")

## 🔍 JSON-LD Context Explained

The `@context` field maps terms to their semantic definitions:

In [ ]:
import json

contexts = {
    "W3C Credentials v2": "https://www.w3.org/ns/credentials/v2",
    "UNTP DPP 0.6.1": "https://test.uncefact.org/vocabulary/untp/dpp/0.6.1/",
}

print("JSON-LD Contexts used in DPP:")
print("─" * 50)
for name, url in contexts.items():
    print(f"\n{name}:")
    print(f"  URL: {url}")
    print(f"  Purpose: Defines vocabulary terms for {name.split()[0]} standard")

## 🆔 DID Identifiers

DPPs can use Decentralized Identifiers (DIDs) for issuers:

In [ ]:
did_examples = {
    "did:web": "did:web:example.com:issuers:company-001",
    "did:key": "did:key:z6MkhaXgBZDvotDkL5257faiztiGiC2QtKLGpbnnEGta2doK",
    "HTTPS URL": "https://example.com/issuers/001",
}

print("Valid issuer ID formats:")
for method, example in did_examples.items():
    print(f"  {method}: {example}")

## 💾 Saving Exported JSON-LD

In [ ]:
from pathlib import Path

if result.passport:
    # Export to file
    output_path = Path("exported_dpp.jsonld")
    jsonld = exporter.export(result.passport)
    output_path.write_text(jsonld)
    print(f"✓ Exported to {output_path}")
    print(f"  Size: {output_path.stat().st_size} bytes")

## 🧹 Cleanup

In [ ]:
Path("exported_dpp.jsonld").unlink(missing_ok=True)
print("✓ Cleaned up")

## 🎉 Key Takeaways

- DPPs are W3C Verifiable Credentials with product-specific claims
- JSON-LD provides semantic interoperability
- Use DIDs for decentralized issuer identification
- Export validated passports for downstream systems